In [48]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return (response.choices[0].message.content)

no_condition  = False
no_non_condition_flow = False
filter_flow = []

default_flow = "Basic flow"


In [49]:
def read_file_into_string(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


In [50]:
def write_to_file(filename, s):
    try:
        with open(filename, 'w') as file:
            file.write(s)
        print("Content has been written to", filename)
    except Exception as e:
        print("An error occurred:", e)

In [51]:
file_path_list = [
"D:\GPT-testing\SpecificationData\Adventura\Adventura - view attraction.txt",
"D:\GPT-testing\SpecificationData\Adventura\Adventura- Search Attraction.txt",
"D:\GPT-testing\SpecificationData\Adventura\Adventura - Filter.txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Login.txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Registry.txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Review test.txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\AddQuestion .txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Handbook.txt",
"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Flashcard.txt",
"D:\GPT-testing\SpecificationData\Trello\create boards.txt",
"D:\GPT-testing\SpecificationData\Trello\login.txt",
"D:\GPT-testing\SpecificationData\Trello\create workspace.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\View detail information of a product.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\Edit personal shopping cart.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\Buy product.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\Add a type of product into cart.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\Track order's status.txt",
"D:\GPT-testing\SpecificationData\ShopeeWeb\Rating a product.txt"
]

In [52]:
file_path = file_path_list[7]
print(file_path)

D:\GPT-testing\SpecificationData\MatchaEnglishWebsite\Handbook.txt


In [53]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
usecase = "Use case: "+ read_file_into_string(file_path)
if usecase is not None:
    print(usecase)

Use case: use case id: 
use case name: handbook
objective: This use case allows to see the vocabulary that learner has learned and setup the vocabulary review test.
actor: learner 
Preconditions:
Learner must be logged into the application.

basic flow:
Step 1: Learner selects the "Handbook" tab on the navigation bar.
Step 2:The system displays a list of leaner's vocabulary list. The list of vocabulary is displayed in vertical rows, each word is a horizontal row displaying the word name, word type, word meaning, memorization level and has a checkbox to mark the appearance of the vocabulary in the review test.
Step 3:Learners can scroll the scroll bar to see more vocabulary.

Alternative Flows:
Alternative flow 1: Learner search vocabulary in Handbook.
At step 3 of the basic flow: Learner presses the search bar.
Step 4: Learner enters the word to search.
Step 5: The system filters vocabulary containing characters entered by learners in the search bar.

Alternative flow 2:Learner search 

In [54]:
SYSTEM_PROMPT_extract_flow ="""
Given use case specification flows.
Extract all the contents of flows into json format.
Return the flows in json format.
{
"Flow name":[ contents of corresponding flow],
}
For example: 
{
  "Main flow:": [
    "Step 1: Learner fills in the username field by a valid username that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding username",
    "Step 3: Learner press \"Login\" button",
    "Step 4: System redirects learner to Home page"
  ],
  "Alternative flow 1: Login by email": [
    "At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding email",
    "Go back to step 3 in the basic flow and continue with the steps from step 3"
  ]
}
"""

In [55]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_extract_flow},
    { "role": "user", "content": usecase}
]

gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)
usecase_flow = json.loads(gpt_response)
flows_name = [key for key in usecase_flow]

{
  "Basic flow": [
    "Step 1: Learner selects the \"Handbook\" tab on the navigation bar.",
    "Step 2: The system displays a list of learner's vocabulary list. The list of vocabulary is displayed in vertical rows, each word is a horizontal row displaying the word name, word type, word meaning, memorization level and has a checkbox to mark the appearance of the vocabulary in the review test.",
    "Step 3: Learners can scroll the scroll bar to see more vocabulary."
  ],
  "Alternative flow 1: Learner search vocabulary in Handbook": [
    "At step 3 of the basic flow: Learner presses the search bar.",
    "Step 4: Learner enters the word to search.",
    "Step 5: The system filters vocabulary containing characters entered by learners in the search bar."
  ],
  "Alternative flow 2: Learner search a meaningless vocabulary in Handbook": [
    "At step 3 of the basic flow: Learner presses the search bar.",
    "Step 4: Learner enters a meaningless word to search.",
    "Step 5: The syst

In [56]:
for flow_name in flows_name:
    if 'Basic' in flow_name:
        default_flow = flow_name
    if 'Main' in flow_name:
        default_flow = flow_name

print(default_flow)

Basic flow


In [57]:
SYSTEM_PROMPT_extract_condition ="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the application. 
For each interactive element:
Identify what type of that element (button, icon, card, tab, multiple tabs,scroll, text field, radio buttons, dropdown menu, sliders, switches, dialog, link, form,rating, filter).
If element is used to input/enter data, then determine all the condition of that element which to make make the input valid and the conditions that would render it invalid base on the description of the use case specification. Do not arbitrarily create additional conditions that not mention in the use case flow.
If element is used to select a value, then determine all the value of the element.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": "condition of that element", "value": "value of that element", "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters", "invalid": "below 8 characters or over 30 character"}, "value": [], "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "value": [], "type": "button"}}
"""

#có nên đổi cái code lọc fliter_data không????

In [58]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_extract_condition},
    { "role": "user", "content": str(usecase_flow) }
  ]
gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)
full_elements = json.loads(gpt_response)

{
  "Handbook tab": {
    "condition": {
      "valid": "",
      "invalid": ""
    },
    "value": [],
    "type": "tab"
  },
  "Scroll bar": {
    "condition": {
      "valid": "",
      "invalid": ""
    },
    "value": [],
    "type": "scroll"
  },
  "Search bar": {
    "condition": {
      "valid": "must contain characters",
      "invalid": "empty input"
    },
    "value": [],
    "type": "text field"
  },
  "Vocabulary checkbox": {
    "condition": {
      "valid": "checkbox can be ticked or unticked",
      "invalid": ""
    },
    "value": ["ticked", "unticked"],
    "type": "checkbox"
  }
}


In [59]:
filtered_data = {key: value for key, value in full_elements.items() if value['type'] 
                 not in ['dialog','button', 'tab', 'card','link','icon','scroll bar','scroll']}

In [60]:
for key, value in filtered_data.items():
    print(key)
    print(value)    

Search bar
{'condition': {'valid': 'must contain characters', 'invalid': 'empty input'}, 'value': [], 'type': 'text field'}
Vocabulary checkbox
{'condition': {'valid': 'checkbox can be ticked or unticked', 'invalid': ''}, 'value': ['ticked', 'unticked'], 'type': 'checkbox'}


In [61]:
filter_element_click_only = {key: value for key, value in full_elements.items() 
                             if (value['type'] in ['dialog','button', 'tab', 'card','link','icon','scroll bar','title'] and not value['value'])}

In [62]:
filter_element_select_value = {key: value for key, value in full_elements.items() 
                             if ( value['value'] and value['type'] not in ['button', 'tab', 'card','link','icon','title','dialog'])}

In [63]:
for key, value in filter_element_select_value.items():
    print(key)
    print(value)    

Vocabulary checkbox
{'condition': {'valid': 'checkbox can be ticked or unticked', 'invalid': ''}, 'value': ['ticked', 'unticked'], 'type': 'checkbox'}


In [64]:
filter_element_enter_value = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and value['condition']['valid'] and value['condition']['invalid'] )}


In [65]:
for key, value in filter_element_enter_value.items():
    print(key)
    print(value)    

Search bar
{'condition': {'valid': 'must contain characters', 'invalid': 'empty input'}, 'value': [], 'type': 'text field'}


In [66]:
SYSTEM_PROMPT_get_flow_name ="""
Given use case flows of a feature and a list of interaction elements extracted from that use case flows.
For each given interactive element:
Identify the flow through which the element is interacted in the use case flow and the type of interaction has with the element and who make that interact.
Ignore interactive elements not mentioned in the given list.
Return the flows in json format.
The JSON format should follow the following structure:
{"Name of interactive element":[{"flow 1":[{"action":"interaction 1","by":"actor","at sentence":"sentence mention that interaction element"},{"action":"interaction 2","by":"actor","at sentence":"sentence mention that interaction element"}]}]}
Examples of output json format template: 
{"Username":[{"Alternative flow 1: Login by valid username":[{"action":"enter","by":"user","at sentence":"Step 4: user enter valid username that above 8 and below 10 characters"}]},{"Exception flow 1: Login by invalid username":[{"action":"fill","by":"user","at sentence":"Step 4: user enter invalid username below 8 characters"}]}]}
{"Rating filter":[{"Alternative flow 1: User rating order":[{"action":"skip","by":"user","at sentence":"Step 3: user skip the rating filter select"},{"action":"select","by":"user","at sentence":"Step 10: user rate the order by select the number of star"}]}]}
"""


In [67]:
Element = "Element: "
for key,value in filter_element_enter_value.items():
    # print(key)
    # print(value)
    Element += key+ " ,"
for key,value in filter_element_select_value.items():
    # print(key)
    # print(value)
    Element += key+ " ,"
Element = Element[:-2] 
Element += "."
print(Element)

Element: Search bar ,Vocabulary checkbox.


In [68]:
print(Element + "\nUse case flow:" + str(usecase_flow)  )

Element: Search bar ,Vocabulary checkbox.
Use case flow:{'Basic flow': ['Step 1: Learner selects the "Handbook" tab on the navigation bar.', "Step 2: The system displays a list of learner's vocabulary list. The list of vocabulary is displayed in vertical rows, each word is a horizontal row displaying the word name, word type, word meaning, memorization level and has a checkbox to mark the appearance of the vocabulary in the review test.", 'Step 3: Learners can scroll the scroll bar to see more vocabulary.'], 'Alternative flow 1: Learner search vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters the word to search.', 'Step 5: The system filters vocabulary containing characters entered by learners in the search bar.'], 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters a meaningless word to search.', 'Step 5: The sy

In [69]:
if(len(filtered_data) != 0 ):
  promptExtractCondtition = [
      { "role": "system", "content": SYSTEM_PROMPT_get_flow_name},
      { "role": "user", "content":Element + "\nUse case flow:" + str(usecase_flow) }
    ]
  gpt_response = ask(promptExtractCondtition, client, model)
  print(gpt_response)
  element_actions_flow = json.loads(gpt_response)
  print(element_actions_flow)

{
  "Search bar": [
    {
      "Alternative flow 1: Learner search vocabulary in Handbook": [
        {
          "action": "press",
          "by": "learner",
          "at sentence": "At step 3 of the basic flow: Learner presses the search bar."
        },
        {
          "action": "enter",
          "by": "learner",
          "at sentence": "Step 4: Learner enters the word to search."
        },
        {
          "action": "filter",
          "by": "system",
          "at sentence": "Step 5: The system filters vocabulary containing characters entered by learners in the search bar."
        }
      ]
    },
    {
      "Alternative flow 2: Learner search a meaningless vocabulary in Handbook": [
        {
          "action": "press",
          "by": "learner",
          "at sentence": "At step 3 of the basic flow: Learner presses the search bar."
        },
        {
          "action": "enter",
          "by": "learner",
          "at sentence": "Step 4: Learner enters a meani

In [70]:
non_actions = ['display','show']
non_actors = ['system','System']

In [71]:
if(len(filtered_data) != 0 ):

    for flow_lists_log in element_actions_flow.values():
        for flow_list_element in flow_lists_log:
            flows_to_delete = []
            for flow_name, actions in flow_list_element.items():
                filtered_actions = []
                for action in actions:
                    if(default_flow in flow_name and 'At step' in action['at sentence'] ):
                        continue

                    if (action['action'] not in non_actions and action['by'] not in non_actors): 
                        filtered_actions.append(action)

                flow_list_element[flow_name] = filtered_actions
                if not filtered_actions:
                    flows_to_delete.append(flow_name)
            flows_to_delete = list(set(flows_to_delete))
            for flow_name in flows_to_delete:
                del flow_list_element[flow_name]

    # Convert modified data back to JSON
    modified_json_content = json.dumps(element_actions_flow, indent=2)
    print(modified_json_content)

{
  "Search bar": [
    {
      "Alternative flow 1: Learner search vocabulary in Handbook": [
        {
          "action": "press",
          "by": "learner",
          "at sentence": "At step 3 of the basic flow: Learner presses the search bar."
        },
        {
          "action": "enter",
          "by": "learner",
          "at sentence": "Step 4: Learner enters the word to search."
        }
      ]
    },
    {
      "Alternative flow 2: Learner search a meaningless vocabulary in Handbook": [
        {
          "action": "press",
          "by": "learner",
          "at sentence": "At step 3 of the basic flow: Learner presses the search bar."
        },
        {
          "action": "enter",
          "by": "learner",
          "at sentence": "Step 4: Learner enters a meaningless word to search."
        }
      ]
    }
  ],
  "Vocabulary checkbox": [
    {
      "Alternative flow 3: Learner set up 1 word to not be review in the review daily test": [
        {
          "a

In [72]:
# for select_element in filter_element_select_value:
#     if(select_element)

In [73]:
# flow_names = []
# for obj in data.values():
#     for flow_name in obj['flow']:
#         flow_names.append(flow_name)

# print(flow_names)
# filter_flow = list(set(flow_names))

In [74]:
print(filter_element_select_value)
print(filter_element_enter_value)
filter_element_select_value_flow_list = []
filter_element_enter_value_flow_list = []
element_flow = element_actions_flow.copy()

{'Vocabulary checkbox': {'condition': {'valid': 'checkbox can be ticked or unticked', 'invalid': ''}, 'value': ['ticked', 'unticked'], 'type': 'checkbox'}}
{'Search bar': {'condition': {'valid': 'must contain characters', 'invalid': 'empty input'}, 'value': [], 'type': 'text field'}}


In [75]:
print(element_flow)

{'Search bar': [{'Alternative flow 1: Learner search vocabulary in Handbook': [{'action': 'press', 'by': 'learner', 'at sentence': 'At step 3 of the basic flow: Learner presses the search bar.'}, {'action': 'enter', 'by': 'learner', 'at sentence': 'Step 4: Learner enters the word to search.'}]}, {'Alternative flow 2: Learner search a meaningless vocabulary in Handbook': [{'action': 'press', 'by': 'learner', 'at sentence': 'At step 3 of the basic flow: Learner presses the search bar.'}, {'action': 'enter', 'by': 'learner', 'at sentence': 'Step 4: Learner enters a meaningless word to search.'}]}], 'Vocabulary checkbox': [{'Alternative flow 3: Learner set up 1 word to not be review in the review daily test': [{'action': 'untick', 'by': 'learner', 'at sentence': 'At step 3 of the basic flow: Learner untick a checkbox of a vocabulary.'}]}, {'Alternative flow 4: Learner set up 1 word to be review in the review daily test': [{'action': 'tick', 'by': 'learner', 'at sentence': 'At step 3 of the

In [76]:
for key in filter_element_select_value:
    if key in element_flow:
        for obj in element_flow[key]:
            filter_element_select_value_flow_list.extend(obj.keys())

    else:
        print(f"No flow found for '{key}' in element_flow.")
filter_element_select_value_flow_list = list(set(filter_element_select_value_flow_list))
print(filter_element_select_value_flow_list)


['Alternative flow 3: Learner set up 1 word to not be review in the review daily test', 'Alternative flow 4: Learner set up 1 word to be review in the review daily test']


In [77]:
for key in filter_element_enter_value:
    if key in element_flow:
        for obj in element_flow[key]:
            filter_element_enter_value_flow_list.extend(obj.keys())

    else:
        print(f"No flow found for '{key}' in element_flow.")

filter_element_enter_value_flow_list = list(set(filter_element_enter_value_flow_list))
print(filter_element_enter_value_flow_list)

['Alternative flow 1: Learner search vocabulary in Handbook', 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook']


In [78]:
# flow with input value


click_only_flow_name = [key for key in flows_name if key not in filter_element_enter_value_flow_list]
click_only_flow_name = [key for key in click_only_flow_name if key not in filter_element_select_value_flow_list]



In [79]:
print(flows_name)
print(filter_element_enter_value_flow_list)
print(filter_element_select_value_flow_list)
print(click_only_flow_name)


['Basic flow', 'Alternative flow 1: Learner search vocabulary in Handbook', 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook', 'Alternative flow 3: Learner set up 1 word to not be review in the review daily test', 'Alternative flow 4: Learner set up 1 word to be review in the review daily test']
['Alternative flow 1: Learner search vocabulary in Handbook', 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook']
['Alternative flow 3: Learner set up 1 word to not be review in the review daily test', 'Alternative flow 4: Learner set up 1 word to be review in the review daily test']
['Basic flow']


In [80]:
click_only_flows = {}

for key,value in usecase_flow.items():
    if key in click_only_flow_name:
        click_only_flows[key] = value

print("click_only_flows = " + str(click_only_flows))


click_only_flows = {'Basic flow': ['Step 1: Learner selects the "Handbook" tab on the navigation bar.', "Step 2: The system displays a list of learner's vocabulary list. The list of vocabulary is displayed in vertical rows, each word is a horizontal row displaying the word name, word type, word meaning, memorization level and has a checkbox to mark the appearance of the vocabulary in the review test.", 'Step 3: Learners can scroll the scroll bar to see more vocabulary.']}


In [81]:
input_flows = {}

for key,value in usecase_flow.items():
    if key in filter_element_enter_value_flow_list:
        input_flows[key] = value

print("input_flows = " + str(input_flows))
print(input_flows.keys())


input_flows = {'Alternative flow 1: Learner search vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters the word to search.', 'Step 5: The system filters vocabulary containing characters entered by learners in the search bar.'], 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters a meaningless word to search.', 'Step 5: The system notifies that there are no suitable search results.']}
dict_keys(['Alternative flow 1: Learner search vocabulary in Handbook', 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook'])


In [82]:
select_flows = {}

for key,value in usecase_flow.items():
    if key in filter_element_select_value_flow_list:
        select_flows[key] = value

print("select_flows = " + str(select_flows))

select_flows = {'Alternative flow 3: Learner set up 1 word to not be review in the review daily test': ['At step 3 of the basic flow: Learner untick a checkbox of a vocabulary.', 'Step 4: The system mark that vocabulary will not appear in the daily test review.'], 'Alternative flow 4: Learner set up 1 word to be review in the review daily test': ['At step 3 of the basic flow: Learner tick a checkbox of a vocabulary.', 'Step 4: The system mark that vocabulary will appear in the daily test review.']}


In [83]:
SYSTEM_PROMPT_gen_no_condition_test_scenarios ="""
Given use case specification flows.
Your task is to generate one test scenario only to test each given flow.
Return the test scenarios generated in json format.
The JSON format should follow the following structure:
{
    "Test Scenarios":[ "name of test scenario"]
  }
For example :
{
  "Test Scenarios": [
    "Verify that users can successfully create a new account with valid credentials."
  ]
}
"""

In [84]:
SYSTEM_PROMPT_gen_test_scenarios_input_element ="""
Given a list of interaction element for input value and their extracted conditions and use case flows that reflect those element.
Your task is to generate a comprehensive set of test scenarios that test all the conditions of given elements.
The test scenarios generated must have clear conditions descritions and mention conditions is tested but do not need example data. 
The positive test scenarios generated must verify the combination of valid elements (the combination follow the description in the use case flow). 
The negative test scenarios generated must verify only one given invalid condition at a time. If there is no invalid condition, you don't need to write negative test scenario.
For cases where the invalid condition does not have a corresponding output description, predict the possible outcome to test.

Return the test scenarios generated in json format.
The JSON format should follow the following structure:
{
  "Positive Test Scenarios": [{ "positive test scenario description": {"conditions": conditions tested}}],
  "Negative Test Scenarios": [{ "negative test scenario description": {"conditions": conditions tested}}]
  }
An example of a test scenario: 
{
  "Positive Test Scenarios": [{ "Verify that users can successfully create a new account with valid username and valid passowrd.": {"conditions":  username valid and password valid}}],
  "Negative Test Scenarios": []
  }
"""

In [85]:
SYSTEM_PROMPT_gen_test_scenarios_select_element ="""
Given a list of element for selecting value, their value options and use case flows use given element.
For each given element:
Generate only one test scenario for each element value options.
Generate test scenarios for the condition (valid, invalid) if element mentions.

Do not generate test scenario to test element that not mention in the given element list.

The JSON format should follow the following structure:
{
    "Test Scenarios":[ "description of scenario 1"]
  }
For example :
{
  "Test Scenarios": [
    "Verify that users can successfully create a new account with valid credentials."
  ]
}
"""
# Generate test scenarios to test the selection for each element's value.


In [86]:
all_test_scenario = []

In [87]:

if( len(click_only_flows)!= 0 ):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_no_condition_test_scenarios},
    { "role": "user", "content": str(click_only_flows) }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(gpt_response)
    temp = json.loads(gpt_response)["Test Scenarios"]
    all_test_scenario.extend(temp)

    

{
  "Test Scenarios": [
    "Verify that learners can view their vocabulary list with all details by selecting the 'Handbook' tab."
  ]
}


In [88]:
print("Element: "+str(filter_element_enter_value) +"\nUse case:" +str(input_flows) )

Element: {'Search bar': {'condition': {'valid': 'must contain characters', 'invalid': 'empty input'}, 'value': [], 'type': 'text field'}}
Use case:{'Alternative flow 1: Learner search vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters the word to search.', 'Step 5: The system filters vocabulary containing characters entered by learners in the search bar.'], 'Alternative flow 2: Learner search a meaningless vocabulary in Handbook': ['At step 3 of the basic flow: Learner presses the search bar.', 'Step 4: Learner enters a meaningless word to search.', 'Step 5: The system notifies that there are no suitable search results.']}


In [89]:

if( len(filter_element_enter_value)!= 0 ):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios_input_element},
    { "role": "user", "content": "Element: "+str(filter_element_enter_value) +"\nUse case:" +str(input_flows)  }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    data = json.loads(gpt_response)

    # Extract only the test scenario names
    all_scenario_names = []
    for scenario_type in data:
        for scenario in data[scenario_type]:
            scenario_desc = list(scenario.keys())[0]
            all_scenario_names.append(scenario_desc)

    print("All Test Scenario Names:")
    print(json.dumps(all_scenario_names, indent=2))
    all_test_scenario.extend(all_scenario_names)
    


All Test Scenario Names:
[
  "Verify that users can successfully search for vocabulary containing specific characters in the Handbook.",
  "Verify that the system notifies there are no suitable search results when a meaningless word is entered in the search bar.",
  "Verify that the system does not proceed with the search when the search bar input is empty."
]


In [90]:
if( len(filter_element_select_value)!= 0):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios_select_element},
    { "role": "user", "content": str(select_flows) + "Element: "+str(filter_element_select_value) }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(gpt_response)
    select_testscenario = json.loads(gpt_response)["Test Scenarios"]
    all_test_scenario.extend(select_testscenario)
    

{
  "Test Scenarios": [
    "Verify that when a learner ticks the vocabulary checkbox, the system marks that vocabulary to appear in the daily test review.",
    "Verify that when a learner unticks the vocabulary checkbox, the system marks that vocabulary will not appear in the daily test review."
  ]
}


In [91]:
print(filter_element_select_value)


{'Vocabulary checkbox': {'condition': {'valid': 'checkbox can be ticked or unticked', 'invalid': ''}, 'value': ['ticked', 'unticked'], 'type': 'checkbox'}}


In [92]:
test_scenario_file_path = file_path.replace("SpecificationData", "ResultSet\\4.21 - best version\\Propse 1\\")
print(test_scenario_file_path)

D:\GPT-testing\ResultSet\4.21 - best version\Propse 1\\MatchaEnglishWebsite\Handbook.txt


In [93]:
for i in (all_test_scenario):
    print(i)

Verify that learners can view their vocabulary list with all details by selecting the 'Handbook' tab.
Verify that users can successfully search for vocabulary containing specific characters in the Handbook.
Verify that the system notifies there are no suitable search results when a meaningless word is entered in the search bar.
Verify that the system does not proceed with the search when the search bar input is empty.
Verify that when a learner ticks the vocabulary checkbox, the system marks that vocabulary to appear in the daily test review.
Verify that when a learner unticks the vocabulary checkbox, the system marks that vocabulary will not appear in the daily test review.


In [94]:
# write_to_file(test_scenario_file_path,"")